<a href="https://colab.research.google.com/github/lee-messi/machine-learning/blob/main/cola_grammar_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow --quiet
!pip install tensorflow-hub --quiet
!pip install tensorflow-text --quiet
!pip install tensorflow-addons --quiet
!pip install tensorflow-datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 17.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.25 which is incompatible.
flax 0.6.11 requires jax>=0.4.2, but you have jax 0.3.25 which is incompatible.
orbax-checkpoint 0.2.6 requires jax>=0.4.9, but you have jax 0.3.25 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 11.6 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

tf.get_logger().setLevel('ERROR')
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
if os.environ['COLAB_TPU_ADDR']:
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  strategy = tf.distribute.TPUStrategy(cluster_resolver)
  print('Using TPU')
elif tf.config.list_physical_devices('GPU'):
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')
else:
  raise ValueError('Running on CPU is not recommended.')

Using TPU


## Import CoLA (The Corpus of Linguistic Acceptability) Data

CoLA is a collection of sentences that has been annotated for acceptability. The sentences are coded/labeled 1 if the sentence is grammatically acceptable or correct, and they are coded 0 if the sentence is grammatically wrong. You can access the publicly available corpus using the link [here](https://nyu-mll.github.io/CoLA/). First, we are going to import the datasets. The datasets are provided in **.tsv** format which can be imported using the **read_csv()** function.

In [4]:
train = pd.read_csv('drive/MyDrive/Colab Notebooks/cola-grammar/in_domain_train.tsv',
                    sep = '\t', header = None)
val = pd.read_csv('drive/MyDrive/Colab Notebooks/cola-grammar/in_domain_dev.tsv',
                  sep = '\t', header = None)
test = pd.read_csv('drive/MyDrive/Colab Notebooks/cola-grammar/out_of_domain_dev.tsv',
                   sep = '\t', header = None)

In [5]:
train.columns = ['misc', 'labels', 'na', 'text']
val.columns = ['misc', 'labels', 'na', 'text']
test.columns = ['misc', 'labels', 'na', 'text']

In [6]:
'{}{}{}'.format(train.shape, val.shape, test.shape)

'(8551, 4)(527, 4)(516, 4)'

First, we are going to inspect the training dataset to ensure that it is balanced - the number of sentences that are grammatically correct and wrong are equal. This is so that the classifier is not biased.

In [7]:
train.labels.value_counts()

1    6023
0    2528
Name: labels, dtype: int64

There are 6,023 sentences with label 1 and 2,528 sentences with label 0. Let's balance these out.

In [8]:
train = train.groupby('labels').sample(1500)

Then, we are going to implement the **df_to_dataset()** function to create a **tf.data.Dataset** using the balanced reviews dataset. This allows us to map the features in the pandas dataframe to features that are more appropriate for training. You can read more about this and check out the function that is used to perform this task [here](https://www.tensorflow.org/tutorials/structured_data/feature_columns). Then, we are going to map the training, validation, and test datasets using the function. Note that depending on the features that you use in the model, you may have to modify parts of the function.

In [9]:
def df_to_dataset(dataframe, shuffle = True, batch_size = 128):
    df = dataframe.copy()
    labels = df.labels
    df = df.text
    ds = tf.data.Dataset.from_tensor_slices((df, labels))
    if shuffle == True:
        ds = ds.shuffle(buffer_size = len(df))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return(ds)

In [10]:
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val)
test_ds = df_to_dataset(test)

## Classifier Model using BERT

In [11]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [25]:
bert_preprocess = hub.KerasLayer(tfhub_handle_preprocess)
bert_encoder = hub.KerasLayer(tfhub_handle_encoder)

In [40]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string)
encoder_input = bert_preprocess(text_input)
encoder_output = bert_encoder(encoder_input)

l = tf.keras.layers.Dense(100, activation = 'relu')(encoder_output['pooled_output'])
l = tf.keras.layers.Dropout(0.3)(l)
l = tf.keras.layers.Dense(50, activation = 'relu')(l)
l = tf.keras.layers.Dense(1, activation = 'sigmoid')(l)
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [41]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = ['accuracy'])

In [42]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_type_ids':   0           ['input_6[0][0]']                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [43]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

In [44]:
history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs = 30,
                    callbacks = [early_stopping])

Epoch 1/30
24/24 [==============================] - 91s 4s/step - loss: 0.6959 - accuracy: 0.5383 - val_loss: 0.6999 - val_accuracy: 0.5446
Epoch 2/30
24/24 [==============================] - 85s 4s/step - loss: 0.6780 - accuracy: 0.5667 - val_loss: 0.6455 - val_accuracy: 0.6490
Epoch 3/30
24/24 [==============================] - 85s 4s/step - loss: 0.6771 - accuracy: 0.5840 - val_loss: 0.6957 - val_accuracy: 0.5085
Epoch 4/30
24/24 [==============================] - 85s 4s/step - loss: 0.6703 - accuracy: 0.5863 - val_loss: 0.6524 - val_accuracy: 0.6471
Epoch 5/30
24/24 [==============================] - 86s 4s/step - loss: 0.6609 - accuracy: 0.5893 - val_loss: 0.6500 - val_accuracy: 0.6433
Epoch 6/30
24/24 [==============================] - 85s 4s/step - loss: 0.6601 - accuracy: 0.6033 - val_loss: 0.6606 - val_accuracy: 0.6376
Epoch 7/30
24/24 [==============================] - 85s 4s/step - loss: 0.6557 - accuracy: 0.6173 - val_loss: 0.5949 - val_accuracy: 0.7002
Epoch 8/30
24/24 [==

## Evaluate Model

In [45]:
model.predict(['he good person is a monster in the woods.'])

1/1 [==============================] - 1s 1s/step


array([[0.44024056]], dtype=float32)

We are going to test out the model using an example sentence that is grammatically wrong. As the prediction value is smaller than 0.5, the model predicted that the sentence is grammatically wrong. This time, we are going to evaluate the model using the test dataset:

In [46]:
model.evaluate(test_ds)

5/5 [==============================] - 13s 2s/step - loss: 0.6089 - accuracy: 0.7054


[0.6089155673980713, 0.7054263353347778]

The model returned an accuracy of **70.54%**. This is a significant improvement from the previous model where the model now ranks around 6th among the models that have been used to perform the identical task on [Kaggle](https://www.kaggle.com/competitions/cola-in-domain-open-evaluation/leaderboard) (although the leaderboard is quite outdated).